<a href="https://colab.research.google.com/github/Bread806/goldbach_backup_from_colab/blob/main/goldbach_1028_MLP_base2357.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/Drive")

Mounted at /content/Drive


In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler


def is_prime(num):
    """檢查一個數字是否為質數"""
    if num <= 1:
        return False
    for i in range(2, int(num ** 0.5) + 1):
        if num % i == 0:
            return False
    return True


def prime_table(x):
    """建立小於x的質數表"""
    primes = [num for num in range(2, x) if is_prime(num)]
    return primes


def convert_base_into_list(number, base, width=10):
    result = []  # init list

    for i in range(width):
        result.append([number % base])
        number = number // base
    result = result[::-1]
    return result


def prime_to_index(primeSize, primes, number):
    for index in range(primeSize):
        if number == primes[index]:
            return index
    return -1

In [ ]:
# shuffle

print("---loading data---")
# load data
trainData = pd.read_csv('traindata_min_size_1000.csv')
trainDataNumbers = trainData['Number'].values
trainDataLabel = trainData['Factors'].values
print("---loading data done.---")

# shuffle training data
shuffleIndices = np.random.permutation(len(trainDataNumbers))
shuffledNumbers = trainDataNumbers[shuffleIndices]
shuffledLabel = trainDataLabel[shuffleIndices]

# collect data

# prepare training set
mergedTrainNumber = []
mergedTrainLabel = []

for num in shuffledNumbers:
    feature = []
    # base(2,3,5,7) and itself

    #itself = [num]
    feature = convert_base_into_list(num, 2)+convert_base_into_list(num, 3)+convert_base_into_list(num, 5)+convert_base_into_list(num, 7)
    #print (feature)
    mergedTrainNumber.append(feature)
    #print(num, " covert to base 2 3 5 7 is done.")

for num in shuffledLabel:
    mergedTrainLabel.append(num)

    #print("merge number ", num, " is done.")

#print (mergedTrainNumber)

mergedArray = np.array(mergedTrainNumber)
mergedLabel = np.array(mergedTrainLabel)

X_train, X_val, y_train, y_val = train_test_split(mergedArray, mergedLabel, test_size=0.2, random_state=42)
#print (X_train[0])

# start training
# 清空所有
keras.clear_session()

################################################################

# require gpu
device = "/device:GPU:0"
with tf.device(device):
    print(f"--- gpu {device} is loaded. ---")
    model = Sequential()

    model.add(Dense(units=40, input_shape=(40,), activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=1, activation='linear'))

    print(model.summary())

    # model.compile(loss='binary_crossentropy', optimizer='adam')   # train version 1
    custom_adam_optimizer = Adam(learning_rate=0.001)
    model.compile(loss='MSE', optimizer=custom_adam_optimizer,
                  metrics=['accuracy'])    # train version 2

    # 训练模型
    print("---starting training---")
    trainHistory = model.fit(X_train, y_train, epochs=200,batch_size=2048, validation_data=(X_val, y_val))
    print("--------------------------------------------------------")

    # save model
    model.save('goldbach_model_1020_400w.h5')
print("model saved.")



---loading data---
---loading data done.---
--- gpu /device:GPU:0 is loaded. ---
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 40)                1640      
                                                                 
 dense_1 (Dense)             (None, 200)               8200      
                                                                 
 dense_2 (Dense)             (None, 200)               40200     
                                                                 
 dense_3 (Dense)             (None, 200)               40200     
                                                                 
 dense_4 (Dense)             (None, 200)               40200     
                                                                 
 dense_5 (Dense)             (None, 1)                 201       
                                         

KeyboardInterrupt: ignored

# predict

In [ ]:
from google.colab import drive
drive.mount("/content/Drive")

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [ ]:
# loadding prime table
primeTable = pd.read_csv("/content/Drive/MyDrive/實驗/goldbach/csv/prime_table_5000000.csv")
primeTableNumber = primeTable["primes"].values
print ("---loading primes table---")

---loading primes table---


In [ ]:
## load data
#predict

from tensorflow.keras.models import load_model
import csv
import pandas as pd
import numpy as np

# 指定 HDF5 模型文件的路徑
modelPath = '/content/Drive/MyDrive/實驗/model_save/goldbach_model_1029_400w.h5'

# 使用 load_model
model = load_model(modelPath)

# load test_set and prime data
testDataPath = '/content/Drive/MyDrive/實驗/goldbach/csv/test_set.csv'
testData = pd.read_csv(testDataPath)
testDataNumbers = testData['Number'].values
testDataLabel = testData['Factors'].values

print("---loading test data done.---")


---loading test data done.---


In [ ]:
# shuffle training data
shuffleIndices = np.random.permutation(len(testDataNumbers))
shuffledNumbers = testDataNumbers[shuffleIndices]
shuffledLabel = testDataLabel[shuffleIndices]

In [ ]:
# generate 25 prime table
primeTable25 = primeTableNumber[:25]
print (primeTable25)

[ 2  3  5  7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89
 97]


In [ ]:
# collect data
# prepare training set
mergedTestNumber = []
mergedTestLabel = []
staticFlagX = [316978, 2652438, 3897236, 3226506, 3571052, 3276150]
staticFlagY = [1577, 20090, 15400, 22108, 11879, 29621]

for num in shuffledNumbers:

    # base(2,3,5,7), aim, radom choice 5 in half of prime
    feature = []
    feature = convert_base_into_list(num, 2)+convert_base_into_list(num, 3)+convert_base_into_list(num, 5)+convert_base_into_list(num, 7)
    mergedTestNumber.append(feature)

for num in shuffledLabel:
    mergedTestLabel.append(num)

In [ ]:
mergedArray = np.array(mergedTestNumber)
mergedLabel = np.array(mergedTestLabel)

In [ ]:
evaluation = model.evaluate(mergedArray, mergedLabel)
XNewData = testDataNumbers, testDataLabel
print ("Loss:", evaluation[0])
print ("Accuracy:", evaluation[1])
print ("##########################")
predictions = model.predict(mergedArray)



2/2 [==============================] - 0s 10ms/step - loss: 938376.3125 - accuracy: 0.0000e+00
Loss: 938376.3125
Accuracy: 0.0
##########################
2/2 [==============================] - 0s 5ms/step


In [ ]:
print (f"測試五十筆資料")
roundPredictions = np.round(predictions)
for index, number in enumerate(testDataNumbers):
  print (f"第{index}筆資料為數字:\t{number}\t|正確答案為:{testDataLabel[index]}\t模型預估為{predictions[index]}, 四捨五入到{roundPredictions[index]}")

測試五十筆資料
第0筆資料為數字:	4234636	|正確答案為:16566	模型預估為[15131.159], 四捨五入到[15131.]
第1筆資料為數字:	4010118	|正確答案為:31717	模型預估為[14785.162], 四捨五入到[14785.]
第2筆資料為數字:	4291942	|正確答案為:14594	模型預估為[28454.123], 四捨五入到[28454.]
第3筆資料為數字:	4151108	|正確答案為:14832	模型預估為[15918.118], 四捨五入到[15918.]
第4筆資料為數字:	4914148	|正確答案為:15673	模型預估為[18817.559], 四捨五入到[18818.]
第5筆資料為數字:	4634032	|正確答案為:16346	模型預估為[13842.603], 四捨五入到[13843.]
第6筆資料為數字:	4010462	|正確答案為:13108	模型預估為[21917.992], 四捨五入到[21918.]
第7筆資料為數字:	4665262	|正確答案為:18098	模型預估為[14231.753], 四捨五入到[14232.]
第8筆資料為數字:	4874538	|正確答案為:31284	模型預估為[22044.635], 四捨五入到[22045.]
第9筆資料為數字:	4867410	|正確答案為:42050	模型預估為[15692.532], 四捨五入到[15693.]
第10筆資料為數字:	4769998	|正確答案為:15623	模型預估為[30096.094], 四捨五入到[30096.]
第11筆資料為數字:	4550968	|正確答案為:15886	模型預估為[14324.187], 四捨五入到[14324.]
第12筆資料為數字:	4324986	|正確答案為:28364	模型預估為[13626.349], 四捨五入到[13626.]
第13筆資料為數字:	4306378	|正確答案為:14047	模型預估為[15311.392], 四捨五入到[15311.]
第14筆資料為數字:	4657756	|正確答案為:14997	模型預估為[29469.188], 四捨五入到[29469.]
第15筆資料為數字:	4948220	|正確答案為:22961	模型預估為[1547

In [ ]:
print ("\t正確答案  "  , "\t/" ,"  \t模型預測")
for index, number in enumerate(testDataNumbers):
  print (f"{index}\t{testDataLabel[index]}  \t\t/  \t{roundPredictions[index]} = {testDataLabel[index]/roundPredictions[index]} ")
print ("###########################################################################")

print ("\t正確答案  "  , "\t-" ,"  \t模型預測")
for index, number in enumerate(testDataNumbers):
  print (f"{index}\t{testDataLabel[index]}  \t\t-  \t{roundPredictions[index]} = {testDataLabel[index]-roundPredictions[index]} ")

	正確答案   	/   	模型預測
0	16566  		/  	[15131.] = [1.0948384] 
1	31717  		/  	[14785.] = [2.1452148] 
2	14594  		/  	[28454.] = [0.512898] 
3	14832  		/  	[15918.] = [0.93177533] 
4	15673  		/  	[18818.] = [0.8328728] 
5	16346  		/  	[13843.] = [1.1808134] 
6	13108  		/  	[21918.] = [0.59804726] 
7	18098  		/  	[14232.] = [1.2716414] 
8	31284  		/  	[22045.] = [1.4190973] 
9	42050  		/  	[15693.] = [2.6795387] 
10	15623  		/  	[30096.] = [0.51910555] 
11	15886  		/  	[14324.] = [1.1090478] 
12	28364  		/  	[13626.] = [2.0816088] 
13	14047  		/  	[15311.] = [0.917445] 
14	14997  		/  	[29469.] = [0.5089077] 
15	22961  		/  	[15472.] = [1.4840357] 
16	16070  		/  	[33537.] = [0.47917226] 
17	13302  		/  	[14330.] = [0.9282624] 
18	13996  		/  	[27032.] = [0.51775676] 
19	15871  		/  	[16158.] = [0.9822379] 
20	15845  		/  	[31459.] = [0.50367147] 
21	32717  		/  	[27842.] = [1.1750952] 
22	31204  		/  	[16261.] = [1.9189472] 
23	19410  		/  	[14106.] = [1.3760102] 
24	30918  		/  	[22609.] = 

In [ ]:
def calculate_mse(predictions, targets):
    """
    計算均方誤差 (MSE)

    參數:
    predictions: 預測值的列表或陣列
    targets: 實際值的列表或陣列，與預測值對應

    返回值:
    mse: 均方誤差
    """
    # 確保預測值和實際值的長度相等
    if len(predictions) != len(targets):
        raise ValueError("預測值和實際值的長度不一致")

    # 計算平方誤差
    squared_errors = [(p - t) ** 2 for p, t in zip(predictions, targets)]

    # 計算均方誤差
    mse = sum(squared_errors) / len(predictions)

    return mse

# 使用 LN
mse_result = calculate_mse(roundPredictions, testDataLabel)

# 使用 MLP(old)
#mse_result = calculate_mse(predictions, testDataPartition)


import math
mean = testData['Factors'].mean()

#+N MSE
MSE = int(calculate_mse(roundPredictions, testDataLabel))

#MLP MSE
# MSE = int(calculate_mse(predictions, testDataPartition))

RMSE= math.sqrt(MSE)
errorRate = RMSE/mean

print (f"MSE : {MSE}\nRMSE : {RMSE}\nError Rate : {errorRate}  ->  {round(errorRate,7)*100}%")

MSE : 112846144
RMSE : 10622.906570237734
Error Rate : 0.5049505155663658  ->  50.49505%


<ipython-input-13-6b3d5dfe95c9>:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  MSE = int(calculate_mse(roundPredictions, testDataLabel))
